<a href="https://colab.research.google.com/github/lucia1299/BERTolde/blob/main/FineTuningBIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install seqeval

In [ ]:
!pip install evaluate

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install datasets

In [ ]:
!pip install -U scikit-learn

In [ ]:
import numpy as np
import torch
from datasets import load_from_disk
from transformers import AutoTokenizer, BertTokenizerFast, BertForSequenceClassification, BertForTokenClassification, DataCollatorForTokenClassification, AutoModelForSequenceClassification, AutoModelForTokenClassification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_dataset = load_from_disk('/content/drive/MyDrive/Colab Notebooks/TESI/tryout/BIOtraining')
validation_dataset = load_from_disk('/content/drive/MyDrive/Colab Notebooks/TESI/tryout/BIOevaluating')
test_dataset = load_from_disk('/content/drive/MyDrive/Colab Notebooks/TESI/tryout/BIOtesting')

In [ ]:
tokenizer=AutoTokenizer.from_pretrained('dbmdz/bert-base-italian-cased')

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
example_text=train_dataset[1]
example_text

{'sentence_id': ['2618'],
 'tokens': ['Proprio',
  "l'",
  'esperienza',
  'degli',
  'anni',
  '80',
  'e',
  'dei',
  'primi',
  'anni',
  '90',
  'indica',
  'come',
  'ben',
  "piu'",
  "dell'",
  "entita'",
  'degli',
  'interventi',
  'conti',
  'la',
  'loro',
  "qualita'",
  'e',
  'il',
  'rispetto',
  ',',
  'da',
  'parte',
  'del',
  'Governo',
  ',',
  'degli',
  'impegni',
  'presi',
  '.'],
 'words_ids': ['word_309',
  'word_310',
  'word_311',
  'word_312',
  'word_313',
  'word_314',
  'word_315',
  'word_316',
  'word_317',
  'word_318',
  'word_319',
  'word_320',
  'word_321',
  'word_322',
  'word_323',
  'word_324',
  'word_325',
  'word_326',
  'word_327',
  'word_328',
  'word_329',
  'word_330',
  'word_331',
  'word_332',
  'word_333',
  'word_334',
  'word_335',
  'word_336',
  'word_337',
  'word_338',
  'word_339',
  'word_340',
  'word_341',
  'word_342',
  'word_343',
  'word_344'],
 'tag': [8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
 

In [ ]:
train_dataset.features['tag']

Sequence(feature=ClassLabel(names=['B-cue', 'B-source', 'B-supplement', 'B-content', 'I-cue', 'I-source', 'I-supplement', 'I-content', 'O'], id=None), length=-1, id=None)

In [ ]:
tokenized_input=tokenizer(example_text['tokens'], is_split_into_words=True)
tokens=tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])
word_ids=tokenized_input.word_ids()
word_ids

[None,
 0,
 1,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 14,
 15,
 15,
 16,
 16,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 None]

In [ ]:
tokens=tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])
tokens

['[CLS]',
 'Proprio',
 'l',
 "'",
 'esperienza',
 'degli',
 'anni',
 '80',
 'e',
 'dei',
 'primi',
 'anni',
 '90',
 'indica',
 'come',
 'ben',
 'piu',
 "'",
 'dell',
 "'",
 'enti',
 '##ta',
 "'",
 'degli',
 'interventi',
 'conti',
 'la',
 'loro',
 'qualita',
 "'",
 'e',
 'il',
 'rispetto',
 ',',
 'da',
 'parte',
 'del',
 'Governo',
 ',',
 'degli',
 'impegni',
 'presi',
 '.',
 '[SEP]']

In [ ]:
len(example_text['tag']), len(tokenized_input['input_ids'])

(36, 44)

In [ ]:
def tokenize_and_align_labels(example, label_all_tokens=True):
    tokenized_input = tokenizer(example['tokens'], padding='max_length', truncation=True, return_tensors='pt', is_split_into_words=True)
    labels = []

    for i, label in enumerate(example['tag']):
        word_ids = tokenized_input.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    labels = torch.tensor(labels).reshape(tokenized_input["input_ids"].shape)
    tokenized_input['labels'] = labels

    return tokenized_input

In [ ]:
q = tokenize_and_align_labels(train_dataset[1:2])
print(q)

{'input_ids': tensor([[  102,  5149,   181,  1553,  3926,   483,   578,  4957,   126,   292,
          2352,   578,  4046,  1731,   342,  1599,   851,  1553,   298,  1553,
          5410,   115,  1553,   483,  5876,  5330,   146,   504, 22143,  1553,
           126,   162,  1527,  1307,   203,   439,   136, 10832,  1307,   483,
          7656,  6761,   697,   103,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [ ]:
for token, label in zip(tokenizer.convert_ids_to_tokens(q['input_ids'][0]), (q['labels'][0])):
    print(f"{token:_<40} {label}")

[CLS]___________________________________ -100
Proprio_________________________________ 8
l_______________________________________ 8
'_______________________________________ 8
esperienza______________________________ 8
degli___________________________________ 8
anni____________________________________ 8
80______________________________________ 8
e_______________________________________ 8
dei_____________________________________ 8
primi___________________________________ 8
anni____________________________________ 8
90______________________________________ 8
indica__________________________________ 8
come____________________________________ 8
ben_____________________________________ 8
piu_____________________________________ 8
'_______________________________________ 8
dell____________________________________ 8
'_______________________________________ 8
enti____________________________________ 8
##ta____________________________________ 8
'_______________________________________ 8
degli___

In [ ]:
tokenized_train = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_eval = validation_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_test = test_dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
len(tokenized_train['input_ids'])

956

In [ ]:
for i in range(len(tokenized_train)):
    try:
        example = tokenized_train[i]
        input_batch = {
            'input_ids': example['input_ids'],
            'token_type_ids': example['token_type_ids'],
            'attention_mask': example['attention_mask']
        }
        target_batch = example['labels']

        input_batch_size = len(input_batch['input_ids'])
        target_batch_size = len(target_batch)

        if input_batch_size != target_batch_size:
            raise ValueError(f"Input batch size ({input_batch_size}) does not match target batch size ({target_batch_size}) at index {i}")
    except Exception as e:
        print("Exception occurred:", e)
        break


In [ ]:
print(type(tokenized_train))

<class 'datasets.arrow_dataset.Dataset'>


In [ ]:
tokenized_train.shape

(956, 8)

In [ ]:
tokenized_eval.shape

(240, 8)

In [ ]:
tokenized_test.shape

(299, 8)

In [ ]:
tokenized_eval

Dataset({
    features: ['sentence_id', 'tokens', 'words_ids', 'tag', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 240
})

In [ ]:
num_labels = train_dataset.features['tag'].feature.num_classes
num_labels

9

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-base-italian-xxl-cased", num_labels = num_labels)

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-italian-xxl-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
label_list = train_dataset.features['tag'].feature.names
label_list

['B-cue',
 'B-source',
 'B-supplement',
 'B-content',
 'I-cue',
 'I-source',
 'I-supplement',
 'I-content',
 'O']

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
example = tokenized_train[1]

In [ ]:
labels = [label_list[i] for i in example['tag']]
labels

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def compute_multiclass_metrics(eval_preds):
    pred_logits, labels = eval_preds
    pred_logits = np.argmax(pred_logits, axis=2)

    # Flatten the predictions and labels for sklearn metrics
    flat_predictions = [item for sublist in pred_logits for item in sublist]
    flat_labels = [item for sublist in labels for item in sublist]

    # We remove all the values where the label is -100
    filtered_predictions = []
    filtered_labels = []
    for pred, label in zip(flat_predictions, flat_labels):
        if label != -100:
            filtered_predictions.append(pred)
            filtered_labels.append(label)

    return {
        "precision": precision_score(filtered_labels, filtered_predictions, average='macro', zero_division=1),
        "recall": recall_score(filtered_labels, filtered_predictions, average='macro', zero_division=1),
        "f1": f1_score(filtered_labels, filtered_predictions, average='macro', zero_division=1),
        "accuracy": accuracy_score(filtered_labels, filtered_predictions),
    }

In [ ]:
stored_predictions = []
stored_labels = []

In [ ]:
training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    logging_steps=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_multiclass_metrics,
)

In [ ]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.003300,0.026132,0.998862,0.333333,0.332763,0.996587
2,0.004100,0.025761,0.998862,0.333333,0.332763,0.996587
3,0.004400,0.025787,0.998862,0.333333,0.332763,0.996587


TrainOutput(global_step=360, training_loss=0.08979900339877026, metrics={'train_runtime': 295.7546, 'train_samples_per_second': 9.697, 'train_steps_per_second': 1.217, 'total_flos': 749446525218816.0, 'train_loss': 0.08979900339877026, 'epoch': 3.0})

In [ ]:
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/TESI/tryout/BIOquote_model')

tokenizer.save_pretrained('/content/drive/MyDrive/Colab Notebooks/TESI/tryout/BIOquote_tokenizer')

('/content/drive/MyDrive/Colab Notebooks/TESI/tryout/BIOquote_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/TESI/tryout/BIOquote_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/TESI/tryout/BIOquote_tokenizer/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/TESI/tryout/BIOquote_tokenizer/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/TESI/tryout/BIOquote_tokenizer/tokenizer.json')

In [ ]:
from transformers import pipeline

In [ ]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained('/content/drive/MyDrive/Colab Notebooks/TESI/tryout/BIOquote_model')

In [ ]:
import json

config = json.load(open('/content/drive/MyDrive/Colab Notebooks/TESI/tryout/BIOquote_model/config.json'))

In [ ]:
id2label= {
    str(i):label for i, label in enumerate(label_list)
}

label2id = {
    label:str(i) for i, label in enumerate(label_list)
}

In [ ]:
config['id2label'] = id2label
config['label2id'] = label2id

# json.dump(config, open('/content/drive/MyDrive/Colab Notebooks/TESI/tryout/BIOquote_model/config.json', 'w'))

In [ ]:
nlp = pipeline('ner', model=model_fine_tuned, tokenizer=tokenizer)

example = """
Il 22 aprile 2024, a New York, il CEO John Smith di TechCorp e il CFO Jane Doe di Innovate Inc. hanno annunciato alla Global Tech Conference: 'La nostra missione è innovare e guidare l'industria tecnologica verso il futuro'.
"""

quote_results = nlp (example)

print(quote_results)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity': 'LABEL_8', 'score': 0.99412954, 'index': 1, 'word': 'Il', 'start': 1, 'end': 3}, {'entity': 'LABEL_8', 'score': 0.9940633, 'index': 2, 'word': '22', 'start': 4, 'end': 6}, {'entity': 'LABEL_8', 'score': 0.994242, 'index': 3, 'word': 'aprile', 'start': 7, 'end': 13}, {'entity': 'LABEL_8', 'score': 0.99430907, 'index': 4, 'word': '202', 'start': 14, 'end': 17}, {'entity': 'LABEL_8', 'score': 0.9938704, 'index': 5, 'word': '##4', 'start': 17, 'end': 18}, {'entity': 'LABEL_8', 'score': 0.9943845, 'index': 6, 'word': ',', 'start': 18, 'end': 19}, {'entity': 'LABEL_8', 'score': 0.9941385, 'index': 7, 'word': 'a', 'start': 20, 'end': 21}, {'entity': 'LABEL_8', 'score': 0.9939545, 'index': 8, 'word': 'New', 'start': 22, 'end': 25}, {'entity': 'LABEL_8', 'score': 0.9936691, 'index': 9, 'word': 'York', 'start': 26, 'end': 30}, {'entity': 'LABEL_8', 'score': 0.9943962, 'index': 10, 'word': ',', 'start': 30, 'end': 31}, {'entity': 'LABEL_8', 'score': 0.9942351, 'index': 11, 'word': 'il

In [ ]:
print(type(quote_results))

<class 'list'>


In [ ]:
x=0
y=0

for my_dict in quote_results:
    if my_dict['entity'] == "LABEL_8":
        x += 1
    else:
        y += 1
print(y)

0
